# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe2a0b0b580>
├── 'a' --> tensor([[ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550]])
└── 'x' --> <FastTreeValue 0x7fe2a0a98490>
    └── 'c' --> tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                        [-0.3174,  0.0224,  0.4834, -1.2349],
                        [-2.9745,  1.5065,  0.9915, -1.3574]])

In [4]:
t.a

tensor([[ 1.2280, -2.0976,  0.0581],
        [-1.7179, -1.4724,  0.2550]])

In [5]:
%timeit t.a

66.1 ns ± 0.0401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe2a0b0b580>
├── 'a' --> tensor([[-1.0455,  0.4353,  0.5092],
│                   [ 0.4438,  0.9852, -1.9432]])
└── 'x' --> <FastTreeValue 0x7fe2a0a98490>
    └── 'c' --> tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                        [-0.3174,  0.0224,  0.4834, -1.2349],
                        [-2.9745,  1.5065,  0.9915, -1.3574]])

In [7]:
%timeit t.a = new_value

65 ns ± 0.0637 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2280, -2.0976,  0.0581],
               [-1.7179, -1.4724,  0.2550]]),
    x: Batch(
           c: tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2280, -2.0976,  0.0581],
        [-1.7179, -1.4724,  0.2550]])

In [11]:
%timeit b.a

52.5 ns ± 0.057 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4036, -0.6765,  0.5559],
               [ 0.7980,  0.5273,  1.1415]]),
    x: Batch(
           c: tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.132 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.173 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 308 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 660 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe2a0a92820>
├── 'a' --> tensor([[[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]],
│           
│                   [[ 1.2280, -2.0976,  0.0581],
│                    [-1.7179, -1.4724,  0.2550]]])
└── 'x' --> <FastTreeValue 0x7fe2a0a92160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe1f675b9d0>
├── 'a' --> tensor([[ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550],
│                   [ 1.2280, -2.0976,  0.0581],
│                   [-1.7179, -1.4724,  0.2550]])
└── 'x' --> <FastTreeValue 0x7fe1f6777f40>
    └── 'c' --> tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                        [-0.3174,  0.0224,  0.4834, -1.2349],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 249 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.4023, -0.1893, -1.1364, -0.0367],
                       [-0.3174,  0.0224,  0.4834, -1.2349],
                       [-2.9745,  1.5065,  0.9915, -1.3574]],
              
                      [[ 1.4023, -0.1893, -1.1364, -0.0367],
                       [-0.3174,  0.0224,  0.4834, -1.2349],
                       [-2.9745,  1.5065,  0.9915, -1.3574]],
              
                      [[ 1.4023, -0.1893, -1.1364, -0.0367],
                       [-0.3174,  0.0224,  0.4834, -1.2349],
                       [-2.9745,  1.5065,  0.9915, -1.3574]],
              
                      [[ 1.4023, -0.1893, -1.1364, -0.0367],
                       [-0.3174,  0.0224,  0.4834, -1.2349],
                       [-2.9745,  1.5065,  0.9915, -1.3574]],
              
                      [[ 1.4023, -0.1893, -1.1364, -0.0367],
                       [-0.3174,  0.0224,  0.4834, -1.2349],
                       [-2.9745,  1.5065,  0.9915, -1.3574]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574],
                      [ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574],
                      [ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574],
                      [ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574],
                      [ 1.4023, -0.1893, -1.1364, -0.0367],
                      [-0.3174,  0.0224,  0.4834, -1.2349],
                      [-2.9745,  1.5065,  0.9915, -1.3574],
                      [ 1.4023, -0.1893, -1.1364, -0.0367],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 733 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
